# A Guided Tour of LSST Calexps
<br>Owner(s): **David Shupe** ([@stargaser](https://github.com/LSSTScienceCollaborations/StackClub/issues/new?body=@stargaser))
<br>Updated for DC2 by:   Douglas Tucker
<br>Last Verified to Run: **2021-02-26**
<br>Verified Stack Release: **v21.0.0**

We'll inspect a visit image ``calexp`` object, and then show how a coadd image differs.

### Learning Objectives:

After working through this tutorial you should be able to follow some best practices when working with ``calexp`` (image) objects.

### Logistics
This notebook is intended to be runnable on `lsst-lsp-stable.ncsa.illinois.edu` from a local git clone of https://github.com/LSSTScienceCollaborations/StackClub.

## Set-up

In [ ]:
# This should match the verified version listed at the start of the notebook
! eups list -s lsst_distrib

In [ ]:
from pprint import pprint

In [ ]:
from lsst.daf.persistence import Butler

In [ ]:
import lsst.afw.display as afw_display

## Retrieving and inspecting a calexp

We have more than one choice of data set available at NCSA for this tutorial.  In particular, we have an HSC data set (which was used by the original version of this notebook) and the DESC DC2 Run2.2i data set (which comprises a simulated 300 sq deg patch of 5 years of Rubin LSST WFD data).

Here, we create a Butler for the data set we wish to examine and grab the calexp image we are interested in exploring. 

In the first part of the tutorial, we will examine the calibrated exposure (`calexp`) for a single `visit`.  In the second part of the tutorial we will examine the calibrated exposure (`calexp`) for a coadd `patch`.

In the following cell, we provide some examples for both the HSC and the DC2 data sets.  If you wish to try others, the current best way for Gen2 Butler  repositories is to search within the relevant `datadir`'s (or `coadd_datadir`'s) to see what is available.  There should be better available methods to do this for the Gen3 Butler.  

**<span style="color:red">WARNING</span>:  if switching from running the DC2 example to running the HSC example, you may need to restart the running kernel in between; otherwise, you might encounter an afw Filter error when running the HSC example.**

In [ ]:
#dataset='HSC'
dataset='DC2'

if dataset == 'HSC':
    # Access HSC RC calexp gen2 repository
    datadir = '/datasets/hsc/repo/rerun/RC/v20_0_0_rc1/DM-25349'
    butler = Butler(datadir)

    # Grab a calexp of interest
    # To see what makes up a dataid for a visit, use:  butler.getKeys('calexp')
    dataId = {'filter': 'HSC-Z', 'ccd': 32, 'visit': 38938}
    calexp = butler.get('calexp', **dataId)

    # Access HST RC gen2 coadd gen2 repository
    # this is a data set that has been specially modified to work with this tutorial
    coadd_datadir = '/project/shared/data/with-globular/'
    coadd_butler = Butler(coadd_datadir)
    
    # Grab a deepCoadd tract,patch
    # To see what makes up a dataid for a coadd patch, use:  coadd_butler.getKeys('deepCoadd_calexp')
    coadd_dataId={'filter':'HSC-I', 'tract':9813, 'patch':'4,4'}
    coadd = coadd_butler.get('deepCoadd_calexp', **coadd_dataId)
    
    
elif dataset == 'DC2':
    # Access DC2 calexp gen2 repository
    datadir = '/datasets/DC2/DR6/Run2.2i/patched/2021-02-10/rerun/run2.2i-calexp-v1/'
    butler = Butler(datadir)

    # Grab a calexp of interest
    # To see what makes up a dataid for a visit, use:  butler.getKeys('calexp')
    dataId = {'filter':'i', 'visit': 512055, 'raftName': 'R20', 'detector': 76}
    calexp = butler.get('calexp', **dataId)
    
    # Access DC2 deepCoadd gen2 repository
    coadd_datadir = '/datasets/DC2/DR6/Run2.2i/patched/2021-02-10/rerun/run2.2i-coadd-wfd-dr6-v1'
    coadd_butler = Butler(coadd_datadir)

    # Grab a deepCoadd tract,patch
    # To see what makes up a dataid for a coadd patch, use:  coadd_butler.getKeys('deepCoadd')
    coadd_dataId = {'filter':'i', 'tract': 4851, 'patch': '1,4'}
    coadd = coadd_butler.get('deepCoadd', **coadd_dataId)
    
else:
    msg = "Unrecognized dataset: %s"%dataset
    raise Exception(msg)

## Image planes / pixel data

In terms of pixel data, a calexp contains an image, a mask, and a variance.

Let's see how to access the image.

In [ ]:
calexp.image

To show the pixel data, we will make use of the matplotlib backend to `lsst.afw.display`.

Due to current limitations of this backend, the display must be defined and used in the same code cell, much as matplotlib commands in a notebook must all be in one cell to produce a plot.

In [ ]:
%matplotlib inline

If the entire calexp is displayed, masks will be overlaid. Here we will eschew the mask display by showing only the image.

In [ ]:
display1 = afw_display.Display(frame=1, backend='matplotlib')
display1.scale("asinh", "zscale")
display1.mtv(calexp.image)

To access the pixel values as an array, use the `.array` attribute.

In [ ]:
data = calexp.image.array
data

In [ ]:
data.__class__

Let's list all the methods for our calexp.

In [ ]:
calexp_methods = [m for m in dir(calexp) if not m.startswith('_')]

In [ ]:
calexp_methods

Access the masked Image

In [ ]:
calexp.maskedImage

Access the variance object and the underlying Numpy array

In [ ]:
calexp.variance

In [ ]:
calexp.variance.array

Access the mask and its underlying array

In [ ]:
calexp.mask

In [ ]:
calexp.mask.array

Get the dimensions of the image, mask and variance

In [ ]:
calexp.getDimensions()

The image, maskedImage and Exposure objects in `lsst.afw.display` include information on **LSST pixels**, which are 0-based with an optional offset.

For a calexp these are usually zero.

In [ ]:
calexp.getXY0()

In [ ]:
calexp.getX0(), calexp.getY0()

## Metadata

Access the wcs object

In [ ]:
wcs = calexp.getWcs()
wcs

The WCS object can be used e.g. to convert pixel coordinates into sky coordinates

In [ ]:
wcs.pixelToSky(100.0, 100.0)

Let's try accessing the metadata, and see what (header) keywords we have.

In [ ]:
metadata = calexp.getMetadata()
pprint(metadata.toDict())

In [ ]:
#metadata.get('CCDTEMP')
metadata.get('TELESCOP')

## Better metadata: ExposureInfo and VisitInfo

For many purposes, information about an exposure is obtainable via the ExposureInfo and VisitInfo classes.

In [ ]:
calexp_info = calexp.getInfo()

In [ ]:
visit_info = calexp_info.getVisitInfo()

In [ ]:
[m for m in dir(visit_info) if not m.startswith('_')]

Obtain weather information for this visit

In [ ]:
visit_info.getWeather()

Check if this calexp has a valid polygon.

In [ ]:
calexp_info.hasValidPolygon()

For the HSC data set example, there is a valid polygon, the `polygon` variable in the next cell gets the polygon values.

For the DESC DC2 data set example, however, there is not a valid polygon; so the `polygon` variable in the next cell returns `None`.

In [ ]:
polygon = calexp_info.getValidPolygon()
print(polygon)

The calexp is not a coadd so this method returns False.

In [ ]:
calexp_info.hasCoaddInputs()

Does the calexp contain transmission curve information?

In [ ]:
calexp_info.hasTransmissionCurve()

Does the calexp contain a World Coordinate System?

In [ ]:
calexp_info.hasWcs()

Does the calexp have a detector?

In [ ]:
calexp_info.hasDetector()

In [ ]:
[m for m in dir(calexp_info.getDetector()) if not m.startswith('_')]

Does the calexp have an aperture correction map?

In [ ]:
calexp_info.hasApCorrMap()

Let's get the aperture correction map and print some information about it

In [ ]:
apCorrMap = calexp_info.getApCorrMap()

In [ ]:
for k in apCorrMap.keys():
    print(k, apCorrMap.get(k))

## Image PSF

Check if our calexp has a PSF

In [ ]:
calexp.hasPsf()

In [ ]:
psf = calexp.getPsf()

The PSF object can be used to get a realization of a PSF at a specific point

In [ ]:
from lsst.geom import PointD
psfimage = psf.computeImage(PointD(100.,100.))

Visualize the PSF

In [ ]:
display1 = afw_display.Display(frame=1, backend='matplotlib')
display1.scale('asinh', min=0.0, max=1.e-3, unit='absolute')
display1.mtv(psfimage)

Access the calibration object which can be used to convert instrumental magnitudes to AB magnitudes

In [ ]:
calib = calexp.getPhotoCalib()
calib

## Image cutouts

We can make a cutout from the calexp in our session.

In [ ]:
import lsst.geom as afwGeom
import lsst.afw.image as afwImage

In [ ]:
bbox = afwGeom.Box2I()
bbox.include(afwGeom.Point2I(400,1400))
bbox.include(afwGeom.Point2I(600,1600))

cutout = calexp[bbox]

Notice that when the image is displayed, the pixel values relate to the parent image.

In [ ]:
display1 = afw_display.Display(frame=1, backend='matplotlib')
display1.scale('asinh', 'zscale')
display1.mtv(cutout.image)

The coordinate of the lower-left-hand pixel is XY0.

In [ ]:
cutout.getXY0()

If a cutout was all that was desired from the start, we could have used our BoundingBox together with our Butler to have read in only the cutout.

In [ ]:
cutout_calexp = butler.get('calexp_sub', bbox=bbox, dataId=dataId)
cutout_calexp.getDimensions()

In [ ]:
display1 = afw_display.Display(frame=1, backend='matplotlib')
display1.scale('asinh', 'zscale')
display1.mtv(cutout_calexp.image)

The `clone` method makes a deep copy. The result can be sliced with a BoundingBox

In [ ]:
clone_cutout = calexp.clone()[bbox]

In [ ]:
display1 = afw_display.Display(frame=1, backend='matplotlib')
display1.scale("asinh", "zscale")
display1.mtv(clone_cutout.image)

## Repeat for a coadd

For this last section, we will look at a coadd image.

Earlier, in the section "Retrieving and inspecting a calexp" above, we retrieved a coadd `calexp`.   Let's see here what methods it provides:

In [ ]:
coadd_methods = [m for m in dir(coadd) if not m.startswith('_')]

In [ ]:
coadd_methods

In [ ]:
set(coadd_methods).symmetric_difference(set(calexp_methods))

The result of the `set` command above shows that a calexp and a coadd have the same methods. This is expected, because they are the same class.

In [ ]:
print(calexp.__class__, coadd.__class__)

A `deepCoadd_calexp` and a visit `calexp` differ mainly in the masks and the xy0 value.

In [ ]:
calexp.mask.getMaskPlaneDict()

In [ ]:
coadd.mask.getMaskPlaneDict()

In [ ]:
coadd.getXY0()

Display the coadd with all masks visible.

In [ ]:
display1 = afw_display.Display(frame=1, backend='matplotlib')
display1.scale("asinh", "zscale")
display1.mtv(coadd)

Display the image data only with a zoom and pan to some nice-looking galaxies, to show off our hyperbolic arcsine stretch:

In [ ]:
display1 = afw_display.Display(frame=1, backend='matplotlib')
display1.scale("asinh", "zscale")
display1.mtv(coadd.image)
display1.zoom(16)
if dataset=='HSC':
    display1.pan(18700, 17000)
else:
    display1.pan(5150, 18650)